In [1]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [2]:
# all these urls are breweries' beer lists
urls = ['https://untappd.com/VanishBeer/beer','https://untappd.com/TrilliumBrewing/beer']#, 
#         'https://untappd.com/treehousebrewco/beer',
#         'https://untappd.com/SaltyTurtleBeerCompany/beer', 'https://untappd.com/BearChaseBrewingCompany/beer','https://untappd.com/TexasLeaguer/beer',
#         'https://untappd.com/WillowParkBrewing/beer','https://untappd.com/PipsMeadery/beer','https://untappd.com/EskerHartBrewingCompany/beer',
#         'https://untappd.com/prolyfk/beer','https://untappd.com/Marlobobo/beer', 'https://untappd.com/EmperorsBrewery/beer',
#         'https://untappd.com/SideProject/beer', 'https://untappd.com/troobado/beer', 'https://untappd.com/w/private-press-brewing/472291/beer', 
#         'https://untappd.com/w/fidens-brewing-co/426930/beer', 'https://untappd.com/MindfulAles/beer', 'https://untappd.com/rootandbranchbrewing/beer'
#         'https://untappd.com/WeaverHollowBrewery/beer', 'https://untappd.com/FreakFolkBier/beer', 'https://untappd.com/HorusAles/beer', 
#         'https://untappd.com/CaseyBrewingBlending/beer', 'https://untappd.com/HouseofFermentology/beer', 'https://untappd.com/treehousebrewco/beer', 
#         'https://untappd.com/barclaybrewingco/beer', 'https://untappd.com/Alchemist_Beer/beer', 'https://untappd.com/HillFarmsteadBrewery/beer',
#         'https://untappd.com/monkishbrewing/beer','https://untappd.com/topplinggoliathbrewing/beer', 'https://untappd.com/OldNationBrewingCompany/beer',
#         'https://untappd.com/PrairieArtisanAles/beer', 'https://untappd.com/fiddlehead/beer', 'https://untappd.com/NewEnglandBrewingCo/beer', 
#         'https://untappd.com/fremontbrewing/beer', 'https://untappd.com/RevisionBrewingCompany/beer', 'https://untappd.com/JackieOsBrewery/beer',
#         'https://untappd.com/westbrookbrewing/beer', 'https://untappd.com/creaturecomforts/beer', 'https://untappd.com/LaCumbreBrewingCompany/beer',
#         'https://untappd.com/SurlyBrewingCompany/beer','https://untappd.com/pfriembeer/beer','https://untappd.com/RevolutionBrewingChicago/beer',
#         'https://untappd.com/NewGlarusBrewingCompany/beer','https://untappd.com/halfacrebeer/beer','https://untappd.com/TheStarkeller/beer',
#         'https://untappd.com/HenHouseBrewing/beer','https://untappd.com/MelvinBrewing/beer','https://untappd.com/AlpineBeerCo/beer',
#         'https://untappd.com/Rhinegeist/beer','https://untappd.com/hardywood/beer', 'https://untappd.com/portcitybrewing/beer'] 

In [3]:
print('Number of different Breweries: ',len(urls))
#I accidenty added a meadery, that can either be good or bad tbh but just noting tgis so I don't forget it

Number of different Breweries:  2


In [8]:
name_ls = []
style_ls = []
desc_ls = []
abv_ls = []
ibu_ls = []

### Show details
scrolls through the page and clicks 'show more' to get the full description

In [9]:
def show_details(items):
    #this loop goes through all the beers in the list and clicks "show more" to get the full description
    for i in range(len(items)):
        beer_det = items[i].find_element_by_css_selector('div.beer-details')
        beer_det.location_once_scrolled_into_view
        try:
            beer_det.find_element_by_css_selector('a.read-more-beerlist.track-click').click()
        except NoSuchElementException:
            continue
        beer_det.location_once_scrolled_into_view

### Get Beer Details
This function goes through the list of beers and gets all the information from them and puts them in a list

In [13]:
def get_beer_details(items):
    for beer in items:   #goes through each beer in the "list" and gets the name, style, IBU, abv, and description (half)
        
        #goes through the beer details and grabs the attributes
        try: 
            name = beer.find_element_by_css_selector('p.name').text    #extracts name of beer
            style = beer.find_element_by_css_selector('p.style').text   #extracts style of beer
            desc = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[4]').text[:-11]   #gets the full description and also cuts off the 'show more'
            
            if (desc=='')|(desc==' '): #if desc isn't long enough to have a "read More" then get the other desc which is stored where the half one is
                desc = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[3]').text[:-11]
                if (desc=='')|(desc==' '):    # in some edge cases the value desc is still going to be blank
                    print('test')
                    desc = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[5]').text[:-11]
            
            if 'no longer being produced' in desc:
                desc = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[6]').text   # if the beer is no longer being produced
            abv = beer.find_element_by_css_selector('div.details-item.abv').text   #extracts abv of beer
            ibu = beer.find_element_by_css_selector('div.details-item.ibu').text   #extracts ibu of beer
            
        # sometimes the namesoace is different for abv and ibu this will catch that
        except NoSuchElementException:
            abv = beer.find_element_by_css_selector('p.abv').text   # part of the way through the process the namespace 
            ibu = beer.find_element_by_css_selector('p.ibu').text   # changes for ibu and abv so this handles that

        name_ls.append(name)     #these add all the attributes to a list that will eventually be set to a df
        style_ls.append(style)
        desc_ls.append(desc)    
        abv_ls.append(abv)             
        ibu_ls.append(ibu)

        print("Name: ", name,'\nStyle:',style,'\nABV: ',abv,'\nIBU: ',ibu,'\nDescription: ',desc,'\n\n------------\n')

#### Brewery scrape
This cell goes through the list of urls (breweries) and gets their beers

In [23]:
for i in range(len(urls)):   #goes through all the urls/breweries
    url=urls[i]    #sets url to new brewery
    driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver
    driver.get(url)   #activates url (brings website up)
    beers = driver.find_elements_by_css_selector('div.beer-item')     #finds each individual beer and sets it to a "list" of beers
    
    show_details(beers)  #method clicks show more on each beer
    get_beer_details(beers)

C:\Users\612456\AppData\Local\Temp\2\ipykernel_1192\1246772369.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver
C:\Users\612456\AppData\Local\Temp\2\ipykernel_1192\1246772369.py:5: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  beers = driver.find_elements_by_css_selector('div.beer-item')     #finds each individual beer and sets it to a "list" of beers


Name:  Ghost Fleet 
Style: IPA - American 
ABV:  7.1% ABV 
IBU:  N/A IBU 
Description:  This is our house IPA. It is hugely hoppy with fruity aromatics from Amarillo and Simcoe hops. A hazy and unfiltered beer with a smooth finish. 

------------

Name:  Fat Boys 
Style: IPA - Imperial / Double 
ABV:  8.5% ABV 
IBU:  N/A IBU 
Description:  This is our Double IPA. It features bold fruity aromatics commonly associate with American IPAs. It contains a ton of hops including Citra and Equinox. As far as DIPAs go this one is light-bodied with an easy finish 

------------

Name:  Wraith 
Style: Wheat Beer - Witbier / Blanche 
ABV:  5.3% ABV 
IBU:  N/A IBU 
Description:  Wraith is a Belgian Style Witbier that provides you with notes of citrus, chamomile and coriander. 

------------

Name:  Lucketts Calling 
Style: IPA - American 
ABV:  6.3% ABV 
IBU:  N/A IBU 
Description:  A hazy IPA with big fruity hop flavors and almost no bitterness. 

------------

Name:  Beyond The Farm 
Style: IPA - A

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: unable to send message to renderer
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x0029B8F3+2406643]
	Ordinal0 [0x0022AF31+1945393]
	Ordinal0 [0x0011C748+837448]
	Ordinal0 [0x0010C9B3+772531]
	Ordinal0 [0x0010CDC8+773576]
	Ordinal0 [0x00118BE0+822240]
	Ordinal0 [0x0010E0BD+778429]
	Ordinal0 [0x0010EC9A+781466]
	Ordinal0 [0x0010E2CA+778954]
	Ordinal0 [0x0010D8A6+776358]
	Ordinal0 [0x0010C7C5+772037]
	Ordinal0 [0x0010CC9D+773277]
	Ordinal0 [0x0011DF4A+843594]
	Ordinal0 [0x00174D3D+1199421]
	Ordinal0 [0x0016427C+1131132]
	Ordinal0 [0x00174682+1197698]
	Ordinal0 [0x00164096+1130646]
	Ordinal0 [0x0013E636+976438]
	Ordinal0 [0x0013F546+980294]
	GetHandleVerifier [0x00509612+2498066]
	GetHandleVerifier [0x004FC920+2445600]
	GetHandleVerifier [0x00334F2A+579370]
	GetHandleVerifier [0x00333D36+574774]
	Ordinal0 [0x00231C0B+1973259]
	Ordinal0 [0x00236688+1992328]
	Ordinal0 [0

#### the above cell takes ~40 mins to run 50 breweries

### Data is recorded in the lists now add to df

In [47]:
#create df
df = pd.DataFrame()
df['Name'] = name_ls
df['style'] = style_ls
df['IBU'] = ibu_ls
df['ABV'] = abv_ls
df['Description'] = desc_ls
df

,Name,style,IBU,ABV,Description
0,Längst (Blend #1),Side Project Brewing,N/A IBU,10% ABV,Längst is our Bourbon barrel-aged Adambier - a...
1,Adam from the Wood,Hair of the Dog Brewing Company,N/A IBU,12% ABV,Adam From The Wood is our Adam aged in America...
2,Cherry Adam from the Wood,Hair of the Dog Brewing Company,N/A IBU,13.5% ABV,Cherry Adam from the Wood is aged with locally...
3,Adam from the Wood - Rye (2015),Hair of the Dog Brewing Company,N/A IBU,12.4% ABV,
4,Bishop's Barrel No. 17,Saint Arnold Brewing Company,30 IBU,13.2% ABV,Adambier Aged in Bourbon Barrels\n\nBishop’s B...
...,...,...,...,...,...
963,CaroBock,Dogfish Head Craft Brewery,25 IBU,8% ABV,"Bavarian-style strong, dark wheat beer aged wi..."
964,Doppeldunkelweizenbock,Rentsch Brewery,25 IBU,9% ABV,Bock -
965,Gefährliches Ding - Brandy Barrel,The Fermentorium Beverage Co.,N/A IBU,9.5% ABV,Bock -
966,Barrel-Aged Tweiss (Beryl Program),Gneiss Brewing Company,N/A IBU,7.5% ABV,"Tweiss, our Weizenbock, aged in french oak bar..."


In [48]:
#get rid of all beers that don't have all 3 major things I need
df = df[(df['IBU']!='N/A IBU')&(df['ABV']!='N/A ABV')&(df['Description']!='')]

In [49]:
df

,Name,style,IBU,ABV,Description
4,Bishop's Barrel No. 17,Saint Arnold Brewing Company,30 IBU,13.2% ABV,Adambier Aged in Bourbon Barrels\n\nBishop’s B...
5,Adam,Hair of the Dog Brewing Company,50 IBU,10% ABV,Adam is a recreation of a historic beerstyle. ...
8,Dortmunder Adambier,Blue Pants Brewery,50 IBU,12.7% ABV,"A recreation of a historical German style, the..."
10,Arcanum (Singularis #1) (Hillrock Estate Soler...,OEC Brewing (Ordinem Ecentrici Coctores),46 IBU,11% ABV,Arcanum is our Adambier. Adambier is an extinc...
11,Thurston Adambier,New Helvetia Brewing Company,18 IBU,12% ABV,Germany has always brewed; and before their la...
...,...,...,...,...,...
960,The Fox and the Stork,Mad Fritz,25 IBU,7.5% ABV,Bock -
962,Weizenbock,Riggs Beer Company,14 IBU,7.3% ABV,Bock -
963,CaroBock,Dogfish Head Craft Brewery,25 IBU,8% ABV,"Bavarian-style strong, dark wheat beer aged wi..."
964,Doppeldunkelweizenbock,Rentsch Brewery,25 IBU,9% ABV,Bock -


In [50]:
df['IBU'] = df['IBU'].str[:-4]

C:\Users\612456\AppData\Local\Temp\1\ipykernel_5700\849542697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IBU'] = df['IBU'].str[:-4]


In [51]:
df['ABV'] = df['ABV'].str[:-5]

C:\Users\612456\AppData\Local\Temp\1\ipykernel_5700\2652638095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ABV'] = df['ABV'].str[:-5]


In [52]:
df['ABV'] = df['ABV'].astype(float)

C:\Users\612456\AppData\Local\Temp\1\ipykernel_5700\2667934893.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ABV'] = df['ABV'].astype(float)


In [53]:
df['IBU'] = df['IBU'].astype(int)

C:\Users\612456\AppData\Local\Temp\1\ipykernel_5700\3279095354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IBU'] = df['IBU'].astype(int)


In [54]:
df

,Name,style,IBU,ABV,Description
4,Bishop's Barrel No. 17,Saint Arnold Brewing Company,30,13.2,Adambier Aged in Bourbon Barrels\n\nBishop’s B...
5,Adam,Hair of the Dog Brewing Company,50,10.0,Adam is a recreation of a historic beerstyle. ...
8,Dortmunder Adambier,Blue Pants Brewery,50,12.7,"A recreation of a historical German style, the..."
10,Arcanum (Singularis #1) (Hillrock Estate Soler...,OEC Brewing (Ordinem Ecentrici Coctores),46,11.0,Arcanum is our Adambier. Adambier is an extinc...
11,Thurston Adambier,New Helvetia Brewing Company,18,12.0,Germany has always brewed; and before their la...
...,...,...,...,...,...
960,The Fox and the Stork,Mad Fritz,25,7.5,Bock -
962,Weizenbock,Riggs Beer Company,14,7.3,Bock -
963,CaroBock,Dogfish Head Craft Brewery,25,8.0,"Bavarian-style strong, dark wheat beer aged wi..."
964,Doppeldunkelweizenbock,Rentsch Brewery,25,9.0,Bock -


OLD CSV

In [ ]:
# save the df to a csv that I can use later and don't have to run this every time
df.to_csv('untappd_beers.csv')

NEW CSV

In [55]:
# save the df to a csv that I can use later and don't have to run this every time
df.to_csv('continuous_beers.csv')

In [240]:
# look at how I can get more beers for 
###Wheat Beer 207 more
###Porter 207 more
###Belgian 203 more
###Lager 191 more
###Pale Ale 178 more
###Wild/Sour Ale 155 more
###Stout 126 more

### new method to get more beers that aren't IPAs

In [42]:
url = 'https://untappd.com/beer/top_rated'
driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver
driver.get(url)   #activates url (brings website up)

C:\Users\612456\AppData\Local\Temp\1\ipykernel_5700\3467423471.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver


In [43]:
name_ls = []
style_ls = []
desc_ls = []
abv_ls = []
ibu_ls = []

selects just the american beers

In [44]:
country = driver.find_element_by_css_selector('select#sort_picker')
country.click()
style = country.find_element_by_xpath('//*[@id="sort_picker"]/option[208]')
style.click()

C:\Users\612456\AppData\Local\Temp\1\ipykernel_5700\421222024.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  country = driver.find_element_by_css_selector('select#sort_picker')


In [45]:
## This is the right idea the names of the drivers are just different at the moment and aren't working
# this will be an easy fix, either change the names of the driver and ther variables below or
# copy the code above and bring it down here and just change the names in that 


# #this goes through each of the styles, clicks on them and skips IPAs
for i in range(2,226):
    item = 'option['+str(i)+']'
    menu = driver.find_element_by_css_selector('select#filter_picker')
    menu.click()  #clicks on the drop down menu
    style = menu.find_element_by_xpath('//*[@id="filter_picker"]/'+item)
    if 'IPA' not in style.text:
        style.click()
        beers = driver.find_elements_by_css_selector('div.beer-item')     #finds each individual beer and sets it to a "list" of beers
        show_details(beers)
        get_beer_details(beers)
    driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)  #scrolls back to the top of the page to click the next style
    element = WebDriverWait(driver, 10).until(   #waits to continue until the drop down menu is selected
        EC.presence_of_element_located((By.ID, "filter_picker"))
    )

C:\Users\612456\AppData\Local\Temp\1\ipykernel_5700\3626706529.py:9: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  menu = driver.find_element_by_css_selector('select#filter_picker')
C:\Users\612456\AppData\Local\Temp\1\ipykernel_5700\3626706529.py:14: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  beers = driver.find_elements_by_css_selector('div.beer-item')     #finds each individual beer and sets it to a "list" of beers


Name:  Längst (Blend #1) 
Style: Side Project Brewing 
ABV:  10% ABV 
IBU:  N/A IBU 
Description:  Längst is our Bourbon barrel-aged Adambier - a historic, strong, malty German ale that was brewed with 7 different German malts and fermented with a fruity yeast strain. Längst was aged for 15 months in a variety of bourbon barrels. Read Less 

------------

test
Name:  Adam from the Wood 
Style: Hair of the Dog Brewing Company 
ABV:  12% ABV 
IBU:  N/A IBU 
Description:  Adam From The Wood is our Adam aged in American Oak barrels. First released in 2000, and released again in November 2011 in 12oz bottles. This 12% beer has lots of the typical HOTD aromas: Caramel, brown sugar, tons of raisin and tobacco. Fig, date, and plum fruitiness in that order. This has a fairly strong earthy vinousness as well as oak vanilla 

------------

Name:  Cherry Adam from the Wood 
Style: Hair of the Dog Brewing Company 
ABV:  13.5% ABV 
IBU:  N/A IBU 
Description:  Cherry Adam from the Wood is aged with 

Name:  The Glass Key 
Style: Cigar City Brewing 
ABV:  9% ABV 
IBU:  N/A IBU 
Description:   

------------

Name:  Baby Schläger 
Style: Against the Grain Brewery 
ABV:  10% ABV 
IBU:  N/A IBU 
Description:  Collaboration with Freigeist. Barrel Aged Smoked Adambie 

------------

Name:  Adambier 
Style: Green Flash Brewing Company 
ABV:  8.3% ABV 
IBU:  N/A IBU 
Description:   

------------

test
Name:  Adambier 
Style: Uinta Brewing Company 
ABV:  10% ABV 
IBU:  N/A IBU 
Description:   

------------

Name:  Adambier 
Style: Alpha Brewing Company 
ABV:  8% ABV 
IBU:  1 IBU 
Description:   

------------

test
Name:  Who The F%&# Is Adam 
Style: Inbound BrewCo 
ABV:  9.1% ABV 
IBU:  10 IBU 
Description:  What the F%&# is an Adambier? This dark, fruity, funky concoction was brewed as a dark dortmunder, kettle soured with lactobacillus brevis, and fermented in oak barrels with ten strains of brettanomyces yeasts. We still have no idea who the F%&# Adam is, though 

------------

Name: 

C:\Users\612456\AppData\Local\Temp\1\ipykernel_5700\3626706529.py:17: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)  #scrolls back to the top of the page to click the next style


test
Name:  Estimation 
Style: Suarez Family Brewery 
ABV:  4.8% ABV 
IBU:  N/A IBU 
Description:  A classic interpretation of the native Dusseldorf style. Burgundy in color and possessing deep notes of raisin, bran flake, and light treacle, yet with a snappy and crisp palate. The malt character is balanced with an amply bitter character achieved with early kettle additions of classic German low-alpha hop varietals 

------------

test
Name:  Stet 
Style: Fox Farm Brewery 
ABV:  5% ABV 
IBU:  N/A IBU 
Description:  Our Düsseldorf-Style Altbier. We’ve expressed our love and admiration for historic brewing cultures before and like Kölsch in nearby Cologne, Altbier has withstood centuries of trends and innovations, to remain the unique standard-bearer for a proud beer drinking city. Brewed with an ale yeast we’re growing increasingly fond of and familiar with and lagered for eight weeks, it delivers malt richness and complexity while remaining crisp and eminently drinkable. Perfect for ki

...

Name:  Confectionator 
Style: Cigar City Brewing 
ABV:  13.5% ABV 
IBU:  46 IBU 
Description:  Rum Barrel-aged Dopplebock with Coconut a 

------------

Name:  Raspberry Eisbock 
Style: Kuhnhenn Brewing Company 
ABV:  15.5% ABV 
IBU:  N/A IBU 
Description:  Boc 

------------

Name:  Barrel Aged Blueberry Eisbock 
Style: Kuhnhenn Brewing Company 
ABV:  15% ABV 
IBU:  N/A IBU 
Description:  Boc 

------------

Name:  Blueberry Eisbock 
Style: Kuhnhenn Brewing Company 
ABV:  15.5% ABV 
IBU:  N/A IBU 
Description:  Boc 

------------

Name:  Tank Bender 
Style: Founders Brewing Co. 
ABV:  15% ABV 
IBU:  N/A IBU 
Description:  Boc 

------------

Name:  Bourbon Barrel Aged Raspberry Eisbock 
Style: Kuhnhenn Brewing Company 
ABV:  15% ABV 
IBU:  N/A IBU 
Description:  Boc 

------------

Name:  30th Anniversary Bourbon Barrel-Aged Eisbock 
Style: Lakefront Brewery 
ABV:  11.16% ABV 
IBU:  25 IBU 
Description:  Boc 

------------

Name:  Texas Ice Bock 
Style: Franconia Brewing Company 
ABV:

Name:  Ziegenmensch 
Style: Schilling Beer Co. 
ABV:  6.5% ABV 
IBU:  N/A IBU 
Description:  Bock - Hell / Maibock  

------------

Name:  Winterhammer 
Style: Schilling Beer Co. 
ABV:  6.9% ABV 
IBU:  N/A IBU 
Description:  Bock - Hell / Maibock  

------------

Name:  Harbinger of Spring 
Style: Folksbier 
ABV:  6% ABV 
IBU:  N/A IBU 
Description:  Brewed with a base of the finest German heirloom barley malt along with German specialty toasted and caramel malts. This heller bock undergoes a long traditional step mash to maximize malt quality before being hopped in the kettle with Spalter Select and a bit of Saphir near the end of the boil. Read Less 

------------

Name:  Prevernal Love 
Style: Wild East Brewing Co. 
ABV:  7% ABV 
IBU:  22 IBU 
Description:  Bock - Hell / Maibock  

------------

Name:  Vernal Genesis 
Style: Nepenthe Brewing Co. 
ABV:  6.9% ABV 
IBU:  N/A IBU 
Description:  Bock - Hell / Maibock  

------------

Name:  Mai-o-maibock 
Style: KC Bier Co. 
ABV:  10% AB

Name:  BBA The Epic Tale of Robert Linzy - Bock 
Style: War Horse Brewing Company 
ABV:  11.9% ABV 
IBU:  N/A IBU 
Description:  Bock - Single /  

------------

Name:  Poop Your Pants 
Style: Perrin Brewing Co. 
ABV:  6% ABV 
IBU:  30 IBU 
Description:  Bock - Single /  

------------

Name:  Imperial Poop Your Pants 
Style: Perrin Brewing Co. 
ABV:  8% ABV 
IBU:  54 IBU 
Description:  Bock - Single /  

------------

Name:  Stagger Bock 
Style: Bonesaw Brewing Co 
ABV:  6.8% ABV 
IBU:  21 IBU 
Description:  Bock - Single /  

------------

Name:  Chromatic Fantasia 
Style: Burning Beard Brewing Company 
ABV:  10% ABV 
IBU:  N/A IBU 
Description:  BBA D 

------------

Name:  Burkhart 
Style: Schilling Beer Co. 
ABV:  7% ABV 
IBU:  N/A IBU 
Description:  Bock - Single /  

------------

Name:  Seipp's Columbia 
Style: The Conrad Seipp Brewing Co. 
ABV:  6% ABV 
IBU:  25 IBU 
Description:  Bock - Single /  

------------

Name:  Winter Daydream 
Style: Kane Brewing Company 
ABV:  7.2% 

Name:  Pacific Gravity 
Style: Firestone Walker Brewing Company 
ABV:  9.2% ABV 
IBU:  N/A IBU 
Description:  Barrel Aged Weizenbock brewed in collaboration with the Pacific Gravity Homebrew Club.Flavors of sweet vanilla and bourbon mingle with banana and clove in this barrel-aged German-style weizenbock. Brewed with wheat, pilsner, Munich, and highly-kilned melanoidin malts for a solid base, PGW has just a touch of hop balance, letting the banana and clove character of our traditional Bavarian wheat beer yeast strain take center stage. Eleven months spent resting in retired charred American oak bourbon barrels adds a layer of complexity reminiscent of Bananas Foster. Read Less 

------------

Name:  Bourbon Barrel La Petite Mort (2014) 
Style: Central Waters Brewing Company 
ABV:  9% ABV 
IBU:  N/A IBU 
Description:  Bourbon Barrel Aged La Petite Mort is a When Central Waters decided to open its brew house for its first collaboration beer, Chicago’s Local Option was the obvious partne

KeyboardInterrupt: 

- The above cell is working, I just need to check out why I got a 
    - WebDriverException: Message: unknown error: session deleted because of page crash
    - from unknown error: cannot determine loading status
    - from tab crashed
    
However:
I got all the way from A-Black and Tan, so honestly I might just create a new csv and continue to add to it, might start at black and tan and continue from there and add to the new csv

Also let's look at just doing beers in america

In [46]:
print(len(name_ls))
print(len(style_ls))
print(len(desc_ls))
print(len(abv_ls))
print(len(ibu_ls))

968
968
968
968
968


In [16]:
# save the df to a csv that I can use later and don't have to run this every time
df.to_csv('untappd_beers.csv')